In [12]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import pandas as pd
from tqdm import tqdm
from pathlib import Path

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o-mini"
DATA_PATH = Path("data/Y-combinator-portfolio.csv")
OUTPUT_PATH = "results/" + DATA_PATH.stem + ".json"
client = OpenAI(api_key=API_KEY)


In [13]:
def load_csv(file_path):
    data = pd.read_csv(file_path)
    return data.to_dict(orient="records")


def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
    return data


# data = load_json(DATA_PATH)
data = load_csv(DATA_PATH)



In [39]:

OUTPUT_FORMAT = """
json
{
"이름": "회사 이름 type: str", 
"현상": "현재 회사에서 풀고있는 문제의 현상 type: list",
"문제": "현재 풀고 있는 회사의 문제 정의 type: list",
"솔루션": "현재 회사에서 제시하는 해결방안 type: str",
"서비스": "현재 회사에서 제공하는 서비스 type: list",
"시장크기": "현재 회사가 목표로 하는 시장의 크기 (단위 1000 달러) type: int",
"한국의 비슷한 서비스": "한국에서 비슷한 서비스를 제공하는 회사들 type: list",
"매출액": "현재 회사 매출액 (단위 1000 달러) type: int",
"시가총액": "현재 회사 시가총액 (단위 1000 달러) type: int",
}
"""

INPUT_PROMPT= f"""
너는 전문 기업 분석가야. 내가 Y-combinator의 포트폴리오 데이터를 입력하면 아래의 OUTPUT FORMAT에 맞는 키워드들을 분석해서 추출해줘.
OUTPUT FORMAT: \n{OUTPUT_FORMAT}
"""

In [42]:
results = []

for item in tqdm(data[:]):
    
    item_str = json.dumps(item, ensure_ascii=False)
    response = client.chat.completions.create(
        model=MODEL,
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": INPUT_PROMPT,
            },
            {"role": "user", "content": item_str},
        ],
    )

    message = response.choices[0].message.content
    parsed_message = json.loads(message)
    results.append(parsed_message)
    
    print(parsed_message)


  0%|          | 1/4687 [00:02<3:38:10,  2.79s/it]

{'이름': 'Airbnb', '현상': ['전 세계에서 숙소 예약의 어려움', '여행지 선택의 제약', '가격 비교의 복잡함'], '문제': ['숙소 예약 플랫폼의 부족', '전통적인 호텔의 높은 가격', '개인 숙소 이용의 불편함'], '솔루션': 'Airbnb는 개인과 호스트를 연결하여 다양한 가격대의 숙소를 제공하며, 편리한 예약 시스템을 통해 여행자에게 저렴하고 다양한 옵션을 만날 수 있게 한다.', '서비스': ['숙소 예약', '호스트와 게스트 매칭', '여행지 추천'], '시장크기': 100000, '한국의 비슷한 서비스': ['에어비앤비', '망고부킹', '여기어때'], '매출액': 100000, '시가총액': 8000000}


  0%|          | 2/4687 [00:05<3:26:30,  2.64s/it]

{'이름': 'Instacart', '현상': ['소비자들이 편리하게 식료품을 구매할 수 있는 방법 부족', '바쁜 생활 속에서 장을 볼 시간 부족'], '문제': ['가정에서 식료품 구매의 번거로움', '온라인 식료품 배송 서비스의 접근성과 효율성 부족'], '솔루션': 'Instacart는 소비자들이 온라인으로 식료품을 쉽게 주문하고 빠르게 배송받을 수 있는 플랫폼을 제공하여 이 문제를 해결하고 있습니다.', '서비스': ['식료품 배달 서비스', '픽업 서비스'], '시장크기': 30000, '한국의 비슷한 서비스': ['쿠팡', '마켓컬리', '배달의민족'], '매출액': 5000, '시가총액': 150000}


  0%|          | 3/4687 [00:08<3:43:15,  2.86s/it]

...

{'이름': 'Benchling', '현상': ['생명과학 연구의 비효율성', '구식 소프트웨어 사용으로 인한 데이터 관리 문제'], '문제': ['생명공학 분야에서 실험과 데이터를 효과적으로 관리할 수 있는 툴 부족', '복잡한 협업 과정에서의 정보 공유 및 통합의 어려움'], '솔루션': '생명과학 연구자들을 위해 현대적인 소프트웨어를 제공하여 실험과 데이터를 더 효율적으로 관리할 수 있도록 지원하는 플랫폼.', '서비스': ['생명과학 데이터 관리 솔루션', '실험실 정보 관리 시스템', '협업 및 데이터 공유 플랫폼'], '시장크기': 100000, '한국의 비슷한 서비스': ['이노비즈', '바이오니아', '메디톡스'], '매출액': 50000, '시가총액': 300000}


  1%|          | 25/4687 [01:13<3:41:25,  2.85s/it]

{'이름': 'Rappi', '현상': ['온디맨드 배달 서비스의 수요 증가', '금융 서비스에 대한 접근성 부족'], '문제': ['소비자들이 필요한 상품과 서비스를 신속하게 받지 못함', '라틴 아메리카에서의 금융 서비스 이용 제한'], '솔루션': '온디맨드 배달 서비스와 금융 서비스를 통합하여 제공', '서비스': ['식료품 배달', '음식 배달', '금융 서비스 제공'], '시장크기': 100000, '한국의 비슷한 서비스': ['배달의민족', '요기요', '쿠팡'], '매출액': 50000, '시가총액': 150000}


  1%|          | 25/4687 [01:16<3:57:38,  3.06s/it]


KeyboardInterrupt: 

In [41]:

with open(OUTPUT_PATH, "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)
